# 출력 파서 실습
- 환경변수 설정 및 llm 객체 생성

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)

- 모듈 및 패키지 로딩

In [11]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

- 자료 구조 정의 (pydantic BaseModel을 상속 받음)

In [12]:
class EmailParser(BaseModel):
    sender_name: str = Field(description="이메일 송신자명")
    title: str = Field(description="이메일 제목")
    content: str = Field(description="이메일 내용")

- json 파서

In [13]:
parser = JsonOutputParser(pydantic_object=EmailParser)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender_name": {"description": "이메일 송신자명", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}
```


- 프롬프트 템플릿 작성

In [14]:
prompt_template = PromptTemplate(
    template="이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n",
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_template 

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자명", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'}, template='이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n')

- 템플릿 포멧팅

In [15]:
prompt = prompt_template.format(email="보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.")
print(prompt)

이메일을 읽고 요약해 주세요.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender_name": {"description": "이메일 송신자명", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}
```
보내는 사람: 김철수
제목: 오늘의 회의록
내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.



- llm에 요청을 보내고, 응답 받기

In [16]:
response = llm.invoke(prompt)

- 응답 결과 파싱하기

In [17]:
parsed_output = parser.invoke(response)
print(parsed_output)

{'sender_name': '김철수', 'title': '오늘의 회의록', 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}


- 문제해결 : LCEL로 실행하기
    - LCEL(**LangChain Expression Language**)

In [18]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수": "질문"})

In [22]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수": "질문"})
chain = prompt_template | llm | parser

email = "보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."

response = chain.invoke({"email":email})

In [23]:
response

{'sender_name': '김철수',
 'title': '오늘의 회의록',
 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}

In [20]:
# ✅ 사전 설치 필요: pip install -U langchain langchain-openai python-dotenv

from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# 1. 🔑 환경 변수 로딩 (OPENAI_API_KEY)
load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

# 2. 🧠 사용할 GPT 모델 정의
llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

# 3. 🧾 출력 결과의 구조 정의 (Pydantic 기반)
class EmailParser(BaseModel):
    sender_name: str = Field(description="이메일 송신자 이름")
    title: str = Field(description="이메일 제목")
    content: str = Field(description="이메일 본문 요약")

# 4. 🧰 Output Parser 생성
parser = JsonOutputParser(pydantic_object=EmailParser)

# 5. 🧱 프롬프트 템플릿 구성
prompt_template = PromptTemplate(
    template=(
        "아래 이메일을 읽고 다음 JSON 형식에 맞게 요약하세요.\n"
        "{format_instructions}\n\n"
        "이메일 전문:\n{email}"
    ),
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 6. 🔁 LCEL로 파이프라인 구성
chain = prompt_template | llm | parser

# 7. 📥 실행할 샘플 이메일
sample_email = """
보낸 사람: 홍길동
제목: 회의 일정 안내
내용: 안녕하세요. 다음 주 월요일 오전 10시에 서울 본사 회의실에서 프로젝트 킥오프 회의가 예정되어 있습니다. 모두 참석 바랍니다.
"""

# 8. 🚀 실행
result = chain.invoke({"email": sample_email})

# 9. ✅ 결과 출력 (Pydantic 객체로 반환됨)
print(result)


{'sender_name': '홍길동', 'title': '회의 일정 안내', 'content': '다음 주 월요일 오전 10시에 서울 본사 회의실에서 프로젝트 킥오프 회의가 예정되어 있으며 참석 요청이 있습니다.'}


In [21]:
response

AIMessage(content='{\n    "sender_name": "김철수",\n    "title": "오늘의 회의록",\n    "content": "오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 294, 'total_tokens': 355, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgRq3tYBRKWmePbCeGnHtJ5PE8Wfy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2ebe80a6-2744-44b1-bec9-2c587ea3f5b2-0', usage_metadata={'input_tokens': 294, 'output_tokens': 61, 'total_tokens': 355, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})